# tSNE

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue May 14 15:43:21 2019

@author: kai

v2: Clean up code, set up functions, optimized alpha etc
v1: PCA and tSNE plots, looping
"""

import numpy as np
import pandas as pd
from matplotlib import rcParams
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import pickle
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import os

In [ ]:

datafolder = '../data/spatial_temporal_4_8-16-16-32_64-64-64-64_5272/'
kinvars = pd.read_hdf(datafolder + 'kinvars_10pc.hdf5')
ee = kinvars['endeffector_coords'].values
ee = np.swapaxes(ee, 0, 1)
modelname = 'spatial_temporal_4_8-16-16-32_64-64-64-64_5272'
ff = 'figs/%s/' %modelname

ilayer = 6

mf = pickle.load(open(datafolder + "data_10pc.pkl", "rb"))
l8 = pickle.load(open(datafolder + f"l{ilayer}_10pc.pkl", "rb"))
labels = pickle.load(open(datafolder + "labels_10pc.pkl", "rb"))

char_labels = ['a', 'b', 'c', 'd', 'e', 'g', 'h', 'l', 'm', 'n',
               'o', 'p', 'q', 'r', 's', 'u', 'v', 'w', 'y', 'z']

#random_idx = np.random.permutation(len(mf))[:20]

mfx = mf.reshape(mf.shape[0], -1)
eex = ee.reshape(ee.shape[0], -1)
l8x = l8.reshape(l8.shape[0], -1)

nlayers= 8

try:
    os.makedirs(ff)
except:
    'folder already exists'


In [ ]:

# %% PCA
## MF
#FIRST INCLUDE PCA
def mypca(X):
    pca = PCA(n_components=3)
    pca_result = pca.fit_transform(X)
        
    df = dict()
    df['pca-one'] = pca_result[:,0]
    df['pca-two'] = pca_result[:,1] 
    df['pca-three'] = pca_result[:,2]
    
    print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))
    
    return pca, df

def pcaplot(df, name):
    plt.figure(dpi=275)
    plt.scatter(
        x = df['pca-one'],
        y = df['pca-two'],
        edgecolors = 'none',
        c=19 - labels,
        cmap='tab20_r',
        alpha = 0.80
    )
    cb = plt.colorbar()
    cb.set_ticks(np.linspace(0.5, 19.5, 21))
    cb.set_ticklabels(char_labels)
    plt.title('%s Output per Label, Dim Red by PCA' %name)
    plt.xlabel('PCA 1')
    plt.ylabel('PCA 2')
    plt.savefig('%s%s_pca.png' %(ff,name))
    plt.close()

mfpca, mfdf = mypca(mfx)
pcaplot(mfdf, 'Muscle Spindles')

##EE
eepca, eedf = mypca(eex)
pcaplot(eedf, 'End Effector')

##L8
'''
l8pca, l8df = mypca(l8x)
pcaplot(l8df, 'Layer 8')
'''

for i in range(nlayers):
    lname = "Layer %d" %i
    
    print(lname)
    l = pickle.load(open(datafolder + f"l{i}_10pc.pkl", "rb"))
    lx = l.reshape(l.shape[0], -1)
    
    ltsne, ldf = mypca(lx)
    pcaplot(ldf, lname)
    
# %% TSNE

def mytsne(X):
    pca = PCA(n_components=50)
    pca_result = pca.fit_transform(X)
    print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))
    print(f'Total explained variance: {sum(pca.explained_variance_ratio_)}')

    tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
    tsne_results = tsne.fit_transform(pca_result)
    
    df = dict()
    df['tsne-2d-one'] = tsne_results[:,0]
    df['tsne-2d-two'] = tsne_results[:,1]
    
    return tsne, df

def plottsne(df, name, alpha=0.8):
    plt.figure(dpi=275)
    plt.scatter(
        x = df['tsne-2d-one'],
        y = df['tsne-2d-two'],
        #s=4,
        edgecolors = 'none',
        c=19 - labels,
        cmap='tab20_r',
        alpha = alpha
    )
    cb = plt.colorbar()
    cb.set_ticks(np.linspace(0.5, 19.5, 21))
    cb.set_ticklabels(char_labels[::-1])
    plt.title('%s tSNE' %name)
    plt.xlabel('tsne-pca50-one')
    plt.ylabel('tsne-pca50-two')
    #plt.savefig('%s%s_tsne%d.png' %(ff,name, int(alpha*100)))
    plt.savefig('%s%s_tsne%d.png' %(ff,name))
    plt.close()

mftsne, mftdf = mytsne(mfx)
plottsne(mftdf, 'Muscle Spindles')

##EE
eetsne, eetdf = mytsne(eex)
plottsne(eetdf, 'End Effectors')

# %% LOOP OVER ALL LAYERS

for i in range(nlayers):
    lname = "Layer %d" %i
    
    print(lname)
    l = pickle.load(open(datafolder + f"l{i}_10pc.pkl", "rb"))
    lx = l.reshape(l.shape[0], -1)
    
    ltsne, ldf = mytsne(lx)
    plottsne(ldf, lname)